# ANALISIS DE RIESGOS: Crear dataframe

En este notebook se crea un dataframe que englobe todos los existentes para los diferentes lustros.
Se crea un fichero .csv padre del cual parta los diferentes análisis a realizar

### PREPARACION DE DATASET

In [1]:
import codecs
import pandas as pd
import csv
from os import listdir
from os.path import isfile, join

##### Leyenda de datos

Extraer del fichero de leyendas, la referencia a los nombres de columnas para poder validar posteriormente los datos de los datasets

In [2]:
info_path = 'Datasets/PHA/info/Afilelayout.txt'

Dar formato al .txt descargado para poder realizar su lectura posterior

In [3]:
# Se bypasea este codigo ya que el fichero ya ha sido transformado
if False:
    # Leemos el fichero
    with open(info_path, 'r') as f:
        lines = f.readlines()

    # Transformamos el fichero del modo que sabemos que los datos estan presentados
    # en este para delimitarlo por comas
    new_text = []

    for line in lines:
        # Eliminar espacios iniciales y finales
        line = line.strip()
        word = ''
        new_line = ''

        # Crear palabras según esta esté dentro de la linea
        for ind, ch in enumerate(line[:-1]):
            next_ch = line[ind+1]

            # Tras una letra existe diferentes espacios en blanco hasta la
            # columna siguiente
            if ' ' not in [ch, next_ch]:
                word += ch

            # Añadir ultima letra y espaciar con coma si no se trata de la 
            # descripción de la columna
            if ind < len(line)-2:
                if ch != ' ' and next_ch == ' ':
                    word += ch
                    
                    if ch.isalpha() and line[ind+2].isalpha():
                        word += ' '
                        continue

                    new_line += word + ','
                    word = ''

        # Añadir en la ultima palabra el salto de linea y agregar la linea al texto
        word += next_ch
        new_line += word + '\n'
        
        new_text.append(new_line)


    # Volver a crear el documento
    with open(info_path, 'w') as f:
        f.writelines(new_text)

Crear diccionario para acceder a los datos de la leyenda

In [4]:
# Convertimos la descripción de las columnas en un diccionario para que el acceso sea sencillo y rápido
dic_legend = {}

with open(info_path) as file:
    reader = csv.reader(file)
    next(reader) # Descartamos cabecera
    for row in reader:
        dic_legend[row[0]] = row[1:5]
        
print('[+] Numero de registros: ', len(dic_legend))

[+] Numero de registros:  180


##### Obtener dataframes

Leer y validar todos los ficheros con datos para realizar PHA.

In [5]:
def validate_df(df) -> bool:
    """
    Validar las columnas existentes en el dataframe creado
    """
    for column in df.columns:
        # Acceder al registro de la leyenda
        if not dic_legend.get(column, None):
            return False
    
    return True

In [6]:
data_path = 'Datasets/PHA/'
pha_file = 'PHA_Data.csv'

In [7]:
# Crear lista con los nombres de los datasets existentes
datasets = [data_path + f for f in listdir(data_path) if isfile(join(data_path, f))]

# Crear un dataframe que posteriormente ira albergando el resto de dataframes que se crean
try:
    unused_files = [data_path + pha_file, data_path + 'AcrftSer.txt', data_path + 'Airport.txt']
    # Crear un dataframe que posteriormente ira albergando el resto de dataframes que se crean
    dataset = datasets[0]
    doc = codecs.open(dataset, 'rU')
    df = pd.read_csv(doc, sep='\t', on_bad_lines='skip', low_memory=False)
    
    # Validar dataset
    if not validate_df(df):
        raise AssertionError

    for dataset in datasets[1:]:
        # Los ficheros dataset tienen _ en su texto
        if dataset not in unused_files:
            doc = codecs.open(dataset, 'rU')
            aux_df = pd.read_csv(doc, sep='\t', on_bad_lines='skip', low_memory=False)
            
            # Validar dataset
            if not validate_df(aux_df):
                raise AssertionError
            
            # Añadir nuevos datos al dataframe
            # Se añade los nuevos registros ordenados por columnas
            df = pd.concat([df, aux_df],axis=0)
            del aux_df
            print('[+] Success: ', dataset)
        
except AssertionError as e:
    print('[-] Fichero error: ', dataset)
    print('[-] Error: ', e.__str__())
    del df
    
    doc = codecs.open(dataset, 'rU')
    error_df = pd.read_csv(doc, sep='\t', on_bad_lines='skip', low_memory=False)
    display(error_df)

[+] Success:  Datasets/PHA/a1995_99.txt
[+] Success:  Datasets/PHA/a2000_04.txt
[+] Success:  Datasets/PHA/a2005_09.txt
[+] Success:  Datasets/PHA/a2010_14.txt
[+] Success:  Datasets/PHA/a2015_19.txt
[+] Success:  Datasets/PHA/A2020_25.txt


Una vez validado el dataframe, lo guardamos como .csv

In [8]:
df.to_csv(data_path + pha_file, index = False)